In [1]:
import pandas as pd
import numpy as np
import datetime, time

import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = 100
pd.options.display.precision = 4

In [3]:
def allsatsundays(start_date, end_date):
    day_sat = pd.date_range(start=start_date, end=end_date, 
                         freq='W-SAT').strftime('%Y-%m-%d').tolist()
    day_sun = pd.date_range(start=start_date, end=end_date, 
                         freq='W-SUN').strftime('%Y-%m-%d').tolist()
    days = day_sat+day_sun
    days.sort()
    return days

def get_stock_market_holydays(filename, start_date, end_date):
    df = pd.read_csv(filename)
    kor_days = df['KOR'][df['KOR'].notnull()].values.tolist()
    usa_days = df['USA'][df['USA'].notnull()].values.tolist() # Nan value를 제외한 값을 리스트로 변환

    holydays = allsatsundays(start_date, end_date)
    
    kor_days = sorted(set(kor_days + holydays))
    usa_days = sorted(set(usa_days + holydays))
    
    return kor_days, usa_days

def get_seq_data(filename, holydays):
    df = pd.read_csv(filename)
    df_seq = df[~df['date'].isin(holydays)].copy()
    df_seq.dropna(subset=df.columns[1:], how='all', inplace=True)
    return df_seq

In [4]:
def classify_data(df, col_name): # column 데이터를 9개로 분류
    col_index = df.columns.tolist()
    for i, col in enumerate(col_name):
        mu = df[col].mean()
        sigma = df[col].std()
        df_min = df[col].min()
        df_max = df[col].max()
        cut_div = [df_min, mu - 3*sigma, mu - 2*sigma, mu - sigma, mu - 0.5*sigma, 
                   mu + 0.5*sigma, mu + sigma, mu + 2*sigma, mu + 3*sigma, df_max ]
        ix = i + col_index.index(col)
        df_new = pd.cut(df[col], cut_div, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9])
        df.insert(ix+1, col+'_class', df_new)
        df[[col, col+'_class']].head()
        
    return df

In [5]:
def consecutive_count(df_sr, idx): # series, idx: row index 여기서부터 역산으로 추출
    # 매도(-), 매수(+) 연속일 계산
    if idx >= 29: # 최대 20개까지만 확인
        df_sel = df_sr[idx-29:idx+1][::-1]
    else:
        df_sel = df_sr[:idx+1][::-1]
    sr = df_sel.apply(lambda x : 1 if (x > 0) else -1)
    p_sum = 0
    m_sum = 0
    z_sum = 0
    for i, num in enumerate(sr):
        if (num == 0) & (i == z_sum) :
            z_sum += 1
            continue
        elif (num == 1) & (i == p_sum) :
            p_sum += 1
            continue
        elif (num == -1) & (i == m_sum) :
            m_sum += 1
            continue   
        else :
            break

    if p_sum > 0 : 
        return p_sum
    elif m_sum > 0 :
        return -m_sum
    else :
        return z_sum

In [6]:
def consequtive_days(df, col_names):
    for col in col_names:
        df_temp = df[col].apply(lambda x : 1 if (x > 0) else -1)
        count = []
        for idx in range(len(df_temp)):
            count.append(consecutive_count(df_temp, idx))

        df[col+'_days'] = count  # 매수, 매도 거래 연속일
        
    return df

In [7]:
def get_weight_ratio(df, investors, investor_sum): # 투자자별 매수 매도 비중 비율
    df['total'] = df[investors_sum].abs().sum(axis=1).to_frame()
    for col in investors:
        df[col+'_ratio'] = df[col]/df['total']*10 # magnify
        
    return df    

In [8]:
def get_change_rate(df, col_names): # get _cr value for each column
    for col in col_names:
        df[col+'_cr'] = (df[col] - df[col].shift(1)) / df[col].shift(1)
    
    return df

In [9]:
# fill nan with the preceesing value
def fillna_with_bfill(df, col_names): 
    for col in col_names:
        df[col].fillna(method='ffill', inplace=True) # forward fill, or bfill
    
    return df

In [10]:
def get_dayofweek(df):
    # insert 요일 column: 1: monday, 5: friday
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dayofweek = df['date'].dt.dayofweek + 1 # 1: monday, 5: Friday
    df.insert(1, 'dayofweek', df_dayofweek)
    
    return df

In [11]:
filename = 'https://raw.githubusercontent.com/fasthill/'\
            'factors_affecting_stock_price/main/data/stock_market_holydays/stock_market_holidays.csv'
start_date = '2020-01-01'
end_date = '2022-12-31'

kor_days, usa_days = get_stock_market_holydays(filename, start_date, end_date) # get stock market holydays

### get and manipulate common data

In [12]:
filename_common = '../data/data_common.csv'
df_common = get_seq_data(filename_common, usa_days)

In [13]:
fill_columns =  ['cpi', 'cpi_anticipated', 'cpi_previous',
                 'fed_rate', 'fed_rate_fore', 'fed_rate_prev', 'fed_rate_ann', 'fed_rate_imp', 
                 'bok_rate', 'bok_rate_fore', 'bok_rate_prev', 'bok_rate_ann', 'bok_rate_imp' ]
futures_columns =  ['fu_usa_date', 'op_usa_date', 'qw_usa_day', 
                 'fu_kor_date', 'op_kor_date', 'dw_kor_day']
df_common = fillna_with_bfill(df_common, fill_columns)
df_common[futures_columns] = df_common[futures_columns].replace(np.nan, 0)

In [14]:
df_common.head()

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,spx_cr,spx_f,spx_f_cr,kospi,kospi_cr,kosdaq,kosdaq_cr,wti,wti_cr,dxy,dxy_cr,krw,krw_cr,bond_usa_10,bond_usa_10_cr,bond_usa_2,bond_usa_2_cr,bond_usa_3m,bond_usa_3m_cr,bond_kor_10,bond_kor_10_cr,bond_kor_2,bond_kor_2_cr,vix,vix_cr,f_g_index,sox,sox_cr,cpi,cpi_anticipated,cpi_previous,gold,gold_cr,fed_rate,fed_rate_fore,fed_rate_prev,fed_rate_ann,fed_rate_imp,bok_rate,bok_rate_fore,bok_rate_prev,bok_rate_ann,bok_rate_imp,fu_usa_date,op_usa_date,qw_usa_day,fu_kor_date,op_kor_date,dw_kor_day
1,2020-01-02,9092.19,1.33%,8891.75,1.31%,28868.80,1.16%,28841.0,0.96%,3257.85,0.84%,3259.00,0.66%,2175.17,-1.02%,674.02,0.63%,61.18,-0.24%,96.525,0.49%,1157.35,0.29%,1.877,-2.29%,1.5710,0.00%,1.533,-1.50%,1.638,-2.03%,1.335,-2.20%,12.47,-9.51%,NaN,1887.9,2.07%,2.10%,2.00%,1.80%,1528.95,0.0069,1.75%,1.75%,1.75%,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-03,9020.77,-0.79%,8810.00,-0.92%,28634.88,-0.81%,28602.0,-0.83%,3234.85,-0.71%,3235.50,-0.72%,2176.46,0.06%,669.93,-0.61%,63.05,3.06%,96.521,0.00%,1164.95,0.66%,1.793,-4.46%,1.5326,-2.44%,1.520,-0.85%,1.552,-5.25%,1.290,-3.37%,14.02,12.43%,NaN,1854.0,-1.80%,2.10%,2.00%,1.80%,1552.24,0.0152,1.75%,1.75%,1.75%,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
5,2020-01-06,9071.47,0.56%,8847.50,0.43%,28703.38,0.24%,28642.0,0.14%,3246.28,0.35%,3243.50,0.25%,2155.07,-0.98%,655.31,-2.18%,63.27,0.35%,96.350,-0.18%,1166.94,0.17%,1.809,2.37%,1.5466,2.11%,1.538,0.49%,1.537,-1.47%,1.299,0.70%,13.85,-1.21%,NaN,1834.7,-1.04%,2.10%,2.00%,1.80%,1563.83,0.0075,1.75%,1.75%,1.75%,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
6,2020-01-07,9068.58,-0.03%,8853.00,0.06%,28583.68,-0.42%,28526.0,-0.40%,3237.18,-0.28%,3235.25,-0.25%,2175.54,0.95%,663.44,1.24%,62.70,-0.90%,96.702,0.37%,1167.30,0.03%,1.811,0.10%,1.5384,-0.53%,1.551,0.85%,1.616,5.14%,1.343,3.39%,13.79,-0.43%,NaN,1867.3,1.78%,2.10%,2.00%,1.80%,1595.24,0.0201,1.75%,1.75%,1.75%,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
7,2020-01-08,9129.24,0.67%,8944.50,1.03%,28745.09,0.56%,28770.0,0.86%,3253.05,0.49%,3260.25,0.77%,2151.31,-1.11%,640.94,-3.39%,59.61,-4.93%,96.996,0.30%,1162.25,-0.43%,1.874,3.48%,1.5846,3.00%,1.544,-0.45%,1.628,0.74%,1.322,-1.56%,13.45,-2.47%,NaN,1867.6,0.02%,2.10%,2.00%,1.80%,1557.89,-0.0234,1.75%,1.75%,1.75%,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df_common.dropna(subset=['ixic', 'dji'], inplace=True) # drop empty rows

In [16]:
df_common.tail()

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,spx_cr,spx_f,spx_f_cr,kospi,kospi_cr,kosdaq,kosdaq_cr,wti,wti_cr,dxy,dxy_cr,krw,krw_cr,bond_usa_10,bond_usa_10_cr,bond_usa_2,bond_usa_2_cr,bond_usa_3m,bond_usa_3m_cr,bond_kor_10,bond_kor_10_cr,bond_kor_2,bond_kor_2_cr,vix,vix_cr,f_g_index,sox,sox_cr,cpi,cpi_anticipated,cpi_previous,gold,gold_cr,fed_rate,fed_rate_fore,fed_rate_prev,fed_rate_ann,fed_rate_imp,bok_rate,bok_rate_fore,bok_rate_prev,bok_rate_ann,bok_rate_imp,fu_usa_date,op_usa_date,qw_usa_day,fu_kor_date,op_kor_date,dw_kor_day
919,2022-07-08,11635.31,0.12%,12152.00,0.11%,31339.20,-0.14%,31310.0,-0.18%,3899.38,-0.08%,3901.25,-0.10%,2350.61,0.70%,766.48,1.12%,104.79,2.01%,106.821,-0.13%,1294.68,-0.23%,3.080,2.67%,3.1047,2.66%,1.963,0.77%,3.365,2.25%,3.318,0.97%,24.64,-5.52%,29.0,2618.0,0.47%,8.60%,8.30%,8.30%,1742.35,0.0052,1.75%,1.50%,1.00%,1.0,1.0,1.75,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
922,2022-07-11,11372.60,-2.26%,11884.25,-2.20%,31175.52,-0.52%,31140.0,-0.54%,3854.43,-1.15%,3856.75,-1.14%,2340.27,-0.44%,767.04,0.07%,103.16,-1.20%,107.829,0.94%,1310.81,1.25%,2.993,-3.37%,3.0697,-1.72%,2.061,7.20%,3.385,0.74%,3.308,-0.30%,26.17,6.21%,27.0,2553.7,-2.46%,8.60%,8.30%,8.30%,1733.39,0.0046,1.75%,1.50%,1.00%,1.0,1.0,1.75,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
923,2022-07-12,11264.73,-0.95%,11779.00,-0.89%,30985.02,-0.61%,30966.0,-0.56%,3818.80,-0.92%,3823.75,-0.86%,2317.76,-0.96%,750.78,-2.12%,94.41,-8.48%,107.912,0.08%,1305.21,-0.43%,2.971,-0.74%,3.0512,-0.60%,2.205,6.99%,3.355,-0.89%,3.259,-1.48%,27.29,4.28%,25.0,2558.2,0.18%,8.60%,8.30%,8.30%,1725.42,-0.0038,1.75%,1.50%,1.00%,1.0,1.0,1.75,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
924,2022-07-13,11247.58,-0.15%,11762.25,-0.14%,30771.54,-0.69%,30758.0,-0.67%,3801.78,-0.45%,3804.50,-0.50%,2328.61,0.47%,763.18,1.65%,95.90,1.58%,107.756,-0.14%,1304.73,-0.04%,2.935,-1.18%,3.1485,3.19%,2.376,7.76%,3.303,-1.55%,3.177,-2.52%,26.82,-1.72%,23.0,2577.4,0.75%,9.10%,8.80%,8.60%,1732.08,0.0119,1.75%,1.50%,1.00%,1.0,1.0,2.25,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
925,2022-07-14,11251.18,0.03%,11797.50,0.30%,30630.01,-0.46%,30604.0,-0.50%,3790.38,-0.30%,3793.25,-0.30%,2322.32,-0.27%,766.08,0.38%,96.65,0.78%,108.407,0.60%,1316.41,0.90%,2.958,0.75%,3.1279,-0.65%,2.399,0.97%,3.305,0.06%,3.215,1.20%,26.40,-1.57%,23.0,2626.9,1.92%,9.10%,8.80%,8.60%,1711.78,-0.0009,1.75%,1.50%,1.00%,1.0,1.0,2.25,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
df_common.replace('%', '', regex=True, inplace = True) # delete '%'

In [18]:
df_common.head()

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,spx_cr,spx_f,spx_f_cr,kospi,kospi_cr,kosdaq,kosdaq_cr,wti,wti_cr,dxy,dxy_cr,krw,krw_cr,bond_usa_10,bond_usa_10_cr,bond_usa_2,bond_usa_2_cr,bond_usa_3m,bond_usa_3m_cr,bond_kor_10,bond_kor_10_cr,bond_kor_2,bond_kor_2_cr,vix,vix_cr,f_g_index,sox,sox_cr,cpi,cpi_anticipated,cpi_previous,gold,gold_cr,fed_rate,fed_rate_fore,fed_rate_prev,fed_rate_ann,fed_rate_imp,bok_rate,bok_rate_fore,bok_rate_prev,bok_rate_ann,bok_rate_imp,fu_usa_date,op_usa_date,qw_usa_day,fu_kor_date,op_kor_date,dw_kor_day
1,2020-01-02,9092.19,1.33,8891.75,1.31,28868.80,1.16,28841.0,0.96,3257.85,0.84,3259.00,0.66,2175.17,-1.02,674.02,0.63,61.18,-0.24,96.525,0.49,1157.35,0.29,1.877,-2.29,1.5710,0.00,1.533,-1.50,1.638,-2.03,1.335,-2.20,12.47,-9.51,NaN,1887.9,2.07,2.10,2.00,1.80,1528.95,0.0069,1.75,1.75,1.75,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-03,9020.77,-0.79,8810.00,-0.92,28634.88,-0.81,28602.0,-0.83,3234.85,-0.71,3235.50,-0.72,2176.46,0.06,669.93,-0.61,63.05,3.06,96.521,0.00,1164.95,0.66,1.793,-4.46,1.5326,-2.44,1.520,-0.85,1.552,-5.25,1.290,-3.37,14.02,12.43,NaN,1854.0,-1.80,2.10,2.00,1.80,1552.24,0.0152,1.75,1.75,1.75,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
5,2020-01-06,9071.47,0.56,8847.50,0.43,28703.38,0.24,28642.0,0.14,3246.28,0.35,3243.50,0.25,2155.07,-0.98,655.31,-2.18,63.27,0.35,96.350,-0.18,1166.94,0.17,1.809,2.37,1.5466,2.11,1.538,0.49,1.537,-1.47,1.299,0.70,13.85,-1.21,NaN,1834.7,-1.04,2.10,2.00,1.80,1563.83,0.0075,1.75,1.75,1.75,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
6,2020-01-07,9068.58,-0.03,8853.00,0.06,28583.68,-0.42,28526.0,-0.40,3237.18,-0.28,3235.25,-0.25,2175.54,0.95,663.44,1.24,62.70,-0.90,96.702,0.37,1167.30,0.03,1.811,0.10,1.5384,-0.53,1.551,0.85,1.616,5.14,1.343,3.39,13.79,-0.43,NaN,1867.3,1.78,2.10,2.00,1.80,1595.24,0.0201,1.75,1.75,1.75,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
7,2020-01-08,9129.24,0.67,8944.50,1.03,28745.09,0.56,28770.0,0.86,3253.05,0.49,3260.25,0.77,2151.31,-1.11,640.94,-3.39,59.61,-4.93,96.996,0.30,1162.25,-0.43,1.874,3.48,1.5846,3.00,1.544,-0.45,1.628,0.74,1.322,-1.56,13.45,-2.47,NaN,1867.6,0.02,2.10,2.00,1.80,1557.89,-0.0234,1.75,1.75,1.75,1.0,1.0,0.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_common.dropna(axis=1, how='all', inplace=True)

In [20]:
df_common.tail()

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,spx_cr,spx_f,spx_f_cr,kospi,kospi_cr,kosdaq,kosdaq_cr,wti,wti_cr,dxy,dxy_cr,krw,krw_cr,bond_usa_10,bond_usa_10_cr,bond_usa_2,bond_usa_2_cr,bond_usa_3m,bond_usa_3m_cr,bond_kor_10,bond_kor_10_cr,bond_kor_2,bond_kor_2_cr,vix,vix_cr,f_g_index,sox,sox_cr,cpi,cpi_anticipated,cpi_previous,gold,gold_cr,fed_rate,fed_rate_fore,fed_rate_prev,fed_rate_ann,fed_rate_imp,bok_rate,fu_usa_date,op_usa_date,qw_usa_day,fu_kor_date,op_kor_date,dw_kor_day
919,2022-07-08,11635.31,0.12,12152.00,0.11,31339.20,-0.14,31310.0,-0.18,3899.38,-0.08,3901.25,-0.10,2350.61,0.70,766.48,1.12,104.79,2.01,106.821,-0.13,1294.68,-0.23,3.080,2.67,3.1047,2.66,1.963,0.77,3.365,2.25,3.318,0.97,24.64,-5.52,29.0,2618.0,0.47,8.60,8.30,8.30,1742.35,0.0052,1.75,1.50,1.00,1.0,1.0,1.75,0.0,0.0,0.0,0.0,0.0,0.0
922,2022-07-11,11372.60,-2.26,11884.25,-2.20,31175.52,-0.52,31140.0,-0.54,3854.43,-1.15,3856.75,-1.14,2340.27,-0.44,767.04,0.07,103.16,-1.20,107.829,0.94,1310.81,1.25,2.993,-3.37,3.0697,-1.72,2.061,7.20,3.385,0.74,3.308,-0.30,26.17,6.21,27.0,2553.7,-2.46,8.60,8.30,8.30,1733.39,0.0046,1.75,1.50,1.00,1.0,1.0,1.75,0.0,0.0,0.0,0.0,0.0,0.0
923,2022-07-12,11264.73,-0.95,11779.00,-0.89,30985.02,-0.61,30966.0,-0.56,3818.80,-0.92,3823.75,-0.86,2317.76,-0.96,750.78,-2.12,94.41,-8.48,107.912,0.08,1305.21,-0.43,2.971,-0.74,3.0512,-0.60,2.205,6.99,3.355,-0.89,3.259,-1.48,27.29,4.28,25.0,2558.2,0.18,8.60,8.30,8.30,1725.42,-0.0038,1.75,1.50,1.00,1.0,1.0,1.75,0.0,0.0,0.0,0.0,0.0,0.0
924,2022-07-13,11247.58,-0.15,11762.25,-0.14,30771.54,-0.69,30758.0,-0.67,3801.78,-0.45,3804.50,-0.50,2328.61,0.47,763.18,1.65,95.90,1.58,107.756,-0.14,1304.73,-0.04,2.935,-1.18,3.1485,3.19,2.376,7.76,3.303,-1.55,3.177,-2.52,26.82,-1.72,23.0,2577.4,0.75,9.10,8.80,8.60,1732.08,0.0119,1.75,1.50,1.00,1.0,1.0,2.25,0.0,0.0,0.0,0.0,0.0,0.0
925,2022-07-14,11251.18,0.03,11797.50,0.30,30630.01,-0.46,30604.0,-0.50,3790.38,-0.30,3793.25,-0.30,2322.32,-0.27,766.08,0.38,96.65,0.78,108.407,0.60,1316.41,0.90,2.958,0.75,3.1279,-0.65,2.399,0.97,3.305,0.06,3.215,1.20,26.40,-1.57,23.0,2626.9,1.92,9.10,8.80,8.60,1711.78,-0.0009,1.75,1.50,1.00,1.0,1.0,2.25,0.0,0.0,0.0,0.0,1.0,0.0


### get and manipulate company data

In [21]:
filename_company = '../data/data_hyunmotor.csv'
df_company = get_seq_data(filename_company, kor_days)

In [22]:
# get day of week
df_company = get_dayofweek(df_company)

In [23]:
# get volumne percentage
df_company['vol_percent'] = (df_company['vol'] / df_company['vol'].sum()) * 100

In [24]:
df_company =  df_company[:-1]  # delete last row 
df_company.tail()

,date,dayofweek,retail,retail_cr,retail_days,foreigner,foreigner_cr,foreigner_days,institution,institution_cr,institution_days,financial,financial_cr,financial_days,invtrust,invtrust_cr,invtrust_days,pension,pension_cr,pension_days,privequity,privequity_cr,privequity_days,bank,bank_cr,bank_days,insurance,insurance_cr,insurance_days,financeetc,financeetc_cr,financeetc_days,corporateetc,corporateetc_cr,corporateetc_days,foreigneretc,foreigneretc_cr,foreigneretc_days,rc1_pcr,rc2_pcr,rc3_pcr,rc4_pcr,open,high,low,close,close_cr,vol,vol_cr,high_time,low_time,vol_percent
919,2022-07-08,5,-20479.0,NaN,NaN,15617.0,NaN,NaN,3853.0,NaN,NaN,-1893.0,NaN,NaN,485.0,NaN,NaN,-1307.0,NaN,NaN,6348.0,NaN,NaN,-2.0,NaN,NaN,456.0,NaN,NaN,-234.0,NaN,NaN,1017.0,NaN,NaN,-8.0,NaN,NaN,NaN,NaN,NaN,NaN,177500.0,180500.0,177000.0,178000.0,1.71,147130.0,NaN,NaN,NaN,0.0922
922,2022-07-11,1,-10749.0,NaN,NaN,11123.0,NaN,NaN,-719.0,NaN,NaN,-3528.0,NaN,NaN,4684.0,NaN,NaN,-354.0,NaN,NaN,-3569.0,NaN,NaN,0.0,NaN,NaN,2056.0,NaN,NaN,-6.0,NaN,NaN,348.0,NaN,NaN,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,180000.0,182000.0,179000.0,179000.0,0.56,119200.0,NaN,NaN,NaN,0.0747
923,2022-07-12,2,868.0,NaN,NaN,3282.0,NaN,NaN,-3835.0,NaN,NaN,-4108.0,NaN,NaN,-563.0,NaN,NaN,3325.0,NaN,NaN,-3446.0,NaN,NaN,28.0,NaN,NaN,1048.0,NaN,NaN,-119.0,NaN,NaN,-204.0,NaN,NaN,-111.0,NaN,NaN,NaN,NaN,NaN,NaN,180000.0,181500.0,177000.0,179000.0,0.00,96440.0,NaN,NaN,NaN,0.0605
924,2022-07-13,3,-40414.0,NaN,NaN,18096.0,NaN,NaN,22755.0,NaN,NaN,6795.0,NaN,NaN,6382.0,NaN,NaN,6062.0,NaN,NaN,2539.0,NaN,NaN,0.0,NaN,NaN,956.0,NaN,NaN,21.0,NaN,NaN,-359.0,NaN,NaN,-79.0,NaN,NaN,NaN,NaN,NaN,NaN,180000.0,185000.0,178500.0,183500.0,2.51,172566.0,NaN,NaN,NaN,0.1082
925,2022-07-14,4,13810.0,NaN,NaN,9619.0,NaN,NaN,-23410.0,NaN,NaN,-14077.0,NaN,NaN,949.0,NaN,NaN,-10782.0,NaN,NaN,141.0,NaN,NaN,25.0,NaN,NaN,318.0,NaN,NaN,17.0,NaN,NaN,56.0,NaN,NaN,-74.0,NaN,NaN,NaN,NaN,NaN,NaN,183000.0,184500.0,180500.0,182500.0,-0.54,142369.0,NaN,NaN,NaN,0.0892


In [25]:
investors = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension',
             'privequity', 'bank',  'insurance', 'financeetc', 'corporateetc', 'foreigneretc']
df_company = classify_data(df_company, investors) # column data catagorizing

In [26]:
# get consecutive days with non empty values
df_company = consequtive_days(df_company, investors)

In [27]:
df_company.head()

,date,dayofweek,retail,retail_class,retail_cr,retail_days,foreigner,foreigner_class,foreigner_cr,foreigner_days,institution,institution_class,institution_cr,institution_days,financial,financial_class,financial_cr,financial_days,invtrust,invtrust_class,invtrust_cr,invtrust_days,pension,pension_class,pension_cr,pension_days,privequity,privequity_class,privequity_cr,privequity_days,bank,bank_class,bank_cr,bank_days,insurance,insurance_class,insurance_cr,insurance_days,financeetc,financeetc_class,financeetc_cr,financeetc_days,corporateetc,corporateetc_class,corporateetc_cr,corporateetc_days,foreigneretc,foreigneretc_class,foreigneretc_cr,foreigneretc_days,rc1_pcr,rc2_pcr,rc3_pcr,rc4_pcr,open,high,low,close,close_cr,vol,vol_cr,high_time,low_time,vol_percent
1,2020-01-02,4,21980.0,5,NaN,1,-14050.0,5,NaN,-1,-12570.0,5,NaN,-1,-2598.0,5,NaN,-1,-531.0,5,NaN,-1,-9936.0,4,NaN,-1,-22.0,5,NaN,-1,50.0,5,NaN,1,355.0,5,NaN,1,112.0,5,NaN,1,4600.0,6,NaN,1,40.0,5,NaN,1,NaN,NaN,NaN,NaN,121000.0,121000.0,118000.0,118000.0,-2.07,66205.0,NaN,NaN,NaN,0.0415
2,2020-01-03,5,42651.0,6,NaN,2,-27939.0,4,NaN,-2,-21214.0,4,NaN,-2,-13522.0,4,NaN,-2,-687.0,5,NaN,-2,-3659.0,5,NaN,-2,-1735.0,5,NaN,-2,-43.0,5,NaN,-1,-1553.0,5,NaN,-1,-17.0,5,NaN,-1,6353.0,7,NaN,2,150.0,5,NaN,2,NaN,NaN,NaN,NaN,118500.0,119000.0,115000.0,116000.0,-1.69,133657.0,NaN,NaN,NaN,0.0838
5,2020-01-06,1,2658.0,5,NaN,3,-1144.0,5,NaN,-3,-5078.0,5,NaN,-3,-9243.0,4,NaN,-3,-563.0,5,NaN,-3,1346.0,5,NaN,1,127.0,5,NaN,1,0.0,5,NaN,-2,3266.0,7,NaN,1,-9.0,5,NaN,-2,3571.0,6,NaN,3,-7.0,5,NaN,-1,NaN,NaN,NaN,NaN,116000.0,117000.0,115500.0,116000.0,0.00,38907.0,NaN,NaN,NaN,0.0244
6,2020-01-07,2,13532.0,5,NaN,4,-5817.0,5,NaN,-4,-11405.0,5,NaN,-4,-2705.0,5,NaN,-4,-1738.0,5,NaN,-4,-6291.0,5,NaN,-1,81.0,5,NaN,2,3.0,5,NaN,1,-755.0,5,NaN,-1,0.0,5,NaN,-3,3741.0,6,NaN,4,-51.0,5,NaN,-2,NaN,NaN,NaN,NaN,116500.0,117000.0,115500.0,115500.0,-0.43,78229.0,NaN,NaN,NaN,0.0490
7,2020-01-08,3,17848.0,5,NaN,5,-1415.0,5,NaN,-5,-20996.0,4,NaN,-5,-6117.0,5,NaN,-5,-1014.0,5,NaN,-5,-16321.0,3,NaN,-2,2514.0,6,NaN,3,2.0,5,NaN,2,-3.0,5,NaN,-2,-57.0,5,NaN,-4,4562.0,6,NaN,5,0.0,5,NaN,-3,NaN,NaN,NaN,NaN,115000.0,115500.0,111500.0,112000.0,-3.03,117813.0,NaN,NaN,NaN,0.0739


In [28]:
# get investors amount relative ratio
investors_sum = ['retail', 'foreigner', 'financial', 'invtrust', 'pension',
         'privequity', 'bank',  'insurance', 'financeetc', 'corporateetc', 'foreigneretc']
df_company = get_weight_ratio(df_company, investors, investors_sum)

In [29]:
df_company.head()

,date,dayofweek,retail,retail_class,retail_cr,retail_days,foreigner,foreigner_class,foreigner_cr,foreigner_days,institution,institution_class,institution_cr,institution_days,financial,financial_class,financial_cr,financial_days,invtrust,invtrust_class,invtrust_cr,invtrust_days,pension,pension_class,pension_cr,pension_days,privequity,privequity_class,privequity_cr,privequity_days,bank,bank_class,bank_cr,bank_days,insurance,insurance_class,insurance_cr,insurance_days,financeetc,financeetc_class,financeetc_cr,financeetc_days,corporateetc,corporateetc_class,corporateetc_cr,corporateetc_days,foreigneretc,foreigneretc_class,foreigneretc_cr,foreigneretc_days,rc1_pcr,rc2_pcr,rc3_pcr,rc4_pcr,open,high,low,close,close_cr,vol,vol_cr,high_time,low_time,vol_percent,total,retail_ratio,foreigner_ratio,institution_ratio,financial_ratio,invtrust_ratio,pension_ratio,privequity_ratio,bank_ratio,insurance_ratio,financeetc_ratio,corporateetc_ratio,foreigneretc_ratio
1,2020-01-02,4,21980.0,5,NaN,1,-14050.0,5,NaN,-1,-12570.0,5,NaN,-1,-2598.0,5,NaN,-1,-531.0,5,NaN,-1,-9936.0,4,NaN,-1,-22.0,5,NaN,-1,50.0,5,NaN,1,355.0,5,NaN,1,112.0,5,NaN,1,4600.0,6,NaN,1,40.0,5,NaN,1,NaN,NaN,NaN,NaN,121000.0,121000.0,118000.0,118000.0,-2.07,66205.0,NaN,NaN,NaN,0.0415,54274.0,4.0498,-2.5887,-2.3160,-0.4787,-0.0978,-1.8307,-0.0041,0.0092,0.0654,0.0206,0.8476,0.0074
2,2020-01-03,5,42651.0,6,NaN,2,-27939.0,4,NaN,-2,-21214.0,4,NaN,-2,-13522.0,4,NaN,-2,-687.0,5,NaN,-2,-3659.0,5,NaN,-2,-1735.0,5,NaN,-2,-43.0,5,NaN,-1,-1553.0,5,NaN,-1,-17.0,5,NaN,-1,6353.0,7,NaN,2,150.0,5,NaN,2,NaN,NaN,NaN,NaN,118500.0,119000.0,115000.0,116000.0,-1.69,133657.0,NaN,NaN,NaN,0.0838,98309.0,4.3385,-2.8420,-2.1579,-1.3755,-0.0699,-0.3722,-0.1765,-0.0044,-0.1580,-0.0017,0.6462,0.0153
5,2020-01-06,1,2658.0,5,NaN,3,-1144.0,5,NaN,-3,-5078.0,5,NaN,-3,-9243.0,4,NaN,-3,-563.0,5,NaN,-3,1346.0,5,NaN,1,127.0,5,NaN,1,0.0,5,NaN,-2,3266.0,7,NaN,1,-9.0,5,NaN,-2,3571.0,6,NaN,3,-7.0,5,NaN,-1,NaN,NaN,NaN,NaN,116000.0,117000.0,115500.0,116000.0,0.00,38907.0,NaN,NaN,NaN,0.0244,21934.0,1.2118,-0.5216,-2.3151,-4.2140,-0.2567,0.6137,0.0579,0.0000,1.4890,-0.0041,1.6281,-0.0032
6,2020-01-07,2,13532.0,5,NaN,4,-5817.0,5,NaN,-4,-11405.0,5,NaN,-4,-2705.0,5,NaN,-4,-1738.0,5,NaN,-4,-6291.0,5,NaN,-1,81.0,5,NaN,2,3.0,5,NaN,1,-755.0,5,NaN,-1,0.0,5,NaN,-3,3741.0,6,NaN,4,-51.0,5,NaN,-2,NaN,NaN,NaN,NaN,116500.0,117000.0,115500.0,115500.0,-0.43,78229.0,NaN,NaN,NaN,0.0490,34714.0,3.8981,-1.6757,-3.2854,-0.7792,-0.5007,-1.8122,0.0233,0.0009,-0.2175,0.0000,1.0777,-0.0147
7,2020-01-08,3,17848.0,5,NaN,5,-1415.0,5,NaN,-5,-20996.0,4,NaN,-5,-6117.0,5,NaN,-5,-1014.0,5,NaN,-5,-16321.0,3,NaN,-2,2514.0,6,NaN,3,2.0,5,NaN,2,-3.0,5,NaN,-2,-57.0,5,NaN,-4,4562.0,6,NaN,5,0.0,5,NaN,-3,NaN,NaN,NaN,NaN,115000.0,115500.0,111500.0,112000.0,-3.03,117813.0,NaN,NaN,NaN,0.0739,49853.0,3.5801,-0.2838,-4.2116,-1.2270,-0.2034,-3.2738,0.5043,0.0004,-0.0006,-0.0114,0.9151,0.0000


In [30]:
investors = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension',
             'privequity', 'bank',  'insurance', 'financeetc', 'corporateetc', 'foreigneretc']
df_company = get_change_rate(df_company, investors) # get column change rate data

In [31]:
df_company.iloc[0:5, 45:60]

,corporateetc_days,foreigneretc,foreigneretc_class,foreigneretc_cr,foreigneretc_days,rc1_pcr,rc2_pcr,rc3_pcr,rc4_pcr,open,high,low,close,close_cr,vol
1,1,40.0,5,NaN,1,NaN,NaN,NaN,NaN,121000.0,121000.0,118000.0,118000.0,-2.07,66205.0
2,2,150.0,5,2.7500,2,NaN,NaN,NaN,NaN,118500.0,119000.0,115000.0,116000.0,-1.69,133657.0
5,3,-7.0,5,-1.0467,-1,NaN,NaN,NaN,NaN,116000.0,117000.0,115500.0,116000.0,0.00,38907.0
6,4,-51.0,5,6.2857,-2,NaN,NaN,NaN,NaN,116500.0,117000.0,115500.0,115500.0,-0.43,78229.0
7,5,0.0,5,-1.0000,-3,NaN,NaN,NaN,NaN,115000.0,115500.0,111500.0,112000.0,-3.03,117813.0


In [32]:
df_company.dropna(axis=1, how='all', inplace=True)

In [33]:
df_company.iloc[0:5, 45:60]

,corporateetc_days,foreigneretc,foreigneretc_class,foreigneretc_cr,foreigneretc_days,open,high,low,close,close_cr,vol,vol_percent,total,retail_ratio,foreigner_ratio
1,1,40.0,5,NaN,1,121000.0,121000.0,118000.0,118000.0,-2.07,66205.0,0.0415,54274.0,4.0498,-2.5887
2,2,150.0,5,2.7500,2,118500.0,119000.0,115000.0,116000.0,-1.69,133657.0,0.0838,98309.0,4.3385,-2.8420
5,3,-7.0,5,-1.0467,-1,116000.0,117000.0,115500.0,116000.0,0.00,38907.0,0.0244,21934.0,1.2118,-0.5216
6,4,-51.0,5,6.2857,-2,116500.0,117000.0,115500.0,115500.0,-0.43,78229.0,0.0490,34714.0,3.8981,-1.6757
7,5,0.0,5,-1.0000,-3,115000.0,115500.0,111500.0,112000.0,-3.03,117813.0,0.0739,49853.0,3.5801,-0.2838


### Merge common and company

In [48]:
# 같은 type (str) 으로 형식 변환
df_common.date = df_common.date.astype(str)
df_company.date = df_company.date.astype(str)

In [73]:
df_merge = pd.merge(df_common,df_company, how='right', on='date') # company 기준 날짜로 merge

In [74]:
df_merge.tail()

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,spx_cr,spx_f,spx_f_cr,kospi,kospi_cr,kosdaq,kosdaq_cr,wti,wti_cr,dxy,dxy_cr,krw,krw_cr,bond_usa_10,bond_usa_10_cr,bond_usa_2,bond_usa_2_cr,bond_usa_3m,bond_usa_3m_cr,bond_kor_10,bond_kor_10_cr,bond_kor_2,bond_kor_2_cr,vix,vix_cr,f_g_index,sox,sox_cr,cpi,cpi_anticipated,cpi_previous,gold,gold_cr,fed_rate,fed_rate_fore,fed_rate_prev,fed_rate_ann,fed_rate_imp,bok_rate,fu_usa_date,...,invtrust_cr,invtrust_days,pension,pension_class,pension_cr,pension_days,privequity,privequity_class,privequity_cr,privequity_days,bank,bank_class,bank_cr,bank_days,insurance,insurance_class,insurance_cr,insurance_days,financeetc,financeetc_class,financeetc_cr,financeetc_days,corporateetc,corporateetc_class,corporateetc_cr,corporateetc_days,foreigneretc,foreigneretc_class,foreigneretc_cr,foreigneretc_days,open,high,low,close,close_cr,vol,vol_percent,total,retail_ratio,foreigner_ratio,institution_ratio,financial_ratio,invtrust_ratio,pension_ratio,privequity_ratio,bank_ratio,insurance_ratio,financeetc_ratio,corporateetc_ratio,foreigneretc_ratio
622,2022-07-08,11635.31,0.12,12152.00,0.11,31339.20,-0.14,31310.0,-0.18,3899.38,-0.08,3901.25,-0.10,2350.61,0.70,766.48,1.12,104.79,2.01,106.821,-0.13,1294.68,-0.23,3.080,2.67,3.1047,2.66,1.963,0.77,3.365,2.25,3.318,0.97,24.64,-5.52,29.0,2618.0,0.47,8.60,8.30,8.30,1742.35,0.0052,1.75,1.50,1.00,1.0,1.0,1.75,0.0,...,-2.7384,1,-1307.0,5,-1.3037,-1,6348.0,7,8.8725,2,-2.0,5,-0.9429,-7,456.0,5,-0.3693,2,-234.0,4,12.0000,-5,1017.0,5,1.1684,3,-8.0,5,-0.4667,-2,177500.0,180500.0,177000.0,178000.0,1.71,147130.0,0.0922,47846.0,-4.2802,3.2640,0.8053,-0.3956,0.1014,-0.2732,1.3268,-0.0004,0.0953,-0.0489,0.2126,-0.0017
623,2022-07-11,11372.60,-2.26,11884.25,-2.20,31175.52,-0.52,31140.0,-0.54,3854.43,-1.15,3856.75,-1.14,2340.27,-0.44,767.04,0.07,103.16,-1.20,107.829,0.94,1310.81,1.25,2.993,-3.37,3.0697,-1.72,2.061,7.20,3.385,0.74,3.308,-0.30,26.17,6.21,27.0,2553.7,-2.46,8.60,8.30,8.30,1733.39,0.0046,1.75,1.50,1.00,1.0,1.0,1.75,0.0,...,8.6577,2,-354.0,5,-0.7292,-2,-3569.0,4,-1.5622,-1,0.0,5,-1.0000,-8,2056.0,6,3.5088,3,-6.0,5,-0.9744,-6,348.0,5,-0.6578,4,-5.0,5,-0.3750,-3,180000.0,182000.0,179000.0,179000.0,0.56,119200.0,0.0747,36422.0,-2.9512,3.0539,-0.1974,-0.9686,1.2860,-0.0972,-0.9799,0.0000,0.5645,-0.0016,0.0955,-0.0014
624,2022-07-12,11264.73,-0.95,11779.00,-0.89,30985.02,-0.61,30966.0,-0.56,3818.80,-0.92,3823.75,-0.86,2317.76,-0.96,750.78,-2.12,94.41,-8.48,107.912,0.08,1305.21,-0.43,2.971,-0.74,3.0512,-0.60,2.205,6.99,3.355,-0.89,3.259,-1.48,27.29,4.28,25.0,2558.2,0.18,8.60,8.30,8.30,1725.42,-0.0038,1.75,1.50,1.00,1.0,1.0,1.75,0.0,...,-1.1202,-1,3325.0,5,-10.3927,1,-3446.0,4,-0.0345,-2,28.0,5,inf,1,1048.0,5,-0.4903,4,-119.0,5,18.8333,-7,-204.0,5,-1.5862,-1,-111.0,4,21.2000,-4,180000.0,181500.0,177000.0,179000.0,0.00,96440.0,0.0605,17102.0,0.5075,1.9191,-2.2424,-2.4021,-0.3292,1.9442,-2.0150,0.0164,0.6128,-0.0696,-0.1193,-0.0649
625,2022-07-13,11247.58,-0.15,11762.25,-0.14,30771.54,-0.69,30758.0,-0.67,3801.78,-0.45,3804.50,-0.50,2328.61,0.47,763.18,1.65,95.90,1.58,107.756,-0.14,1304.73,-0.04,2.935,-1.18,3.1485,3.19,2.376,7.76,3.303,-1.55,3.177,-2.52,26.82,-1.72,23.0,2577.4,0.75,9.10,8.80,8.60,1732.08,0.0119,1.75,1.50,1.00,1.0,1.0,2.25,0.0,...,-12.3357,1,6062.0,6,0.8232,2,2539.0,6,-1.7368,1,0.0,5,-1.0000,-1,956.0,5,-0.0878,5,21.0,5,-1.1765,1,-359.0,5,0.7598,-2,-79.0,5,-0.2883,-5,180000.0,185000.0,178500.0,183500.0,2.51,172566.0,0.1082,81703.0,-4.9465,2.2149,2.7851,0.8317,0.7811,0.7420,0.3108,0.0000,0.1170,0.0026,-0.0439,-0.0097
626,2022-07-14,11251.18,0.03,11797.50,0.30,30630.01,-0.46,30604.0,-0.50,3790.38,-0.30,3793.25,-0.30,2322.32,-0.27,766.08,0.38,96.65,0.78,108.407,0.60,1316.41,0.90,2.958,0.75,3.1279,-0.65,2.399,0.97,3.305,0.06,3.215,1.20,26.40,-1.57,23.0,2626.9,1.92,9.10,8.80,8.60,1711.78,-0.0009,1.75,1.50,1.00,1.0,1.0,2.25,0.0,...,-0.8513,2,-10782.0,4,-2.7786,-1,141.0,5,-0.9445,2,25.0,5,inf,1,318.0,5,-0.6674,6,17.0,5,-0.1905,2,56.0,5,-1.1560,1,-74.0,5,-

In [77]:
df_merge[df_merge['ixic'].isna()]

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,spx_cr,spx_f,spx_f_cr,kospi,kospi_cr,kosdaq,kosdaq_cr,wti,wti_cr,dxy,dxy_cr,krw,krw_cr,bond_usa_10,bond_usa_10_cr,bond_usa_2,bond_usa_2_cr,bond_usa_3m,bond_usa_3m_cr,bond_kor_10,bond_kor_10_cr,bond_kor_2,bond_kor_2_cr,vix,vix_cr,f_g_index,sox,sox_cr,cpi,cpi_anticipated,cpi_previous,gold,gold_cr,fed_rate,fed_rate_fore,fed_rate_prev,fed_rate_ann,fed_rate_imp,bok_rate,fu_usa_date,...,invtrust_cr,invtrust_days,pension,pension_class,pension_cr,pension_days,privequity,privequity_class,privequity_cr,privequity_days,bank,bank_class,bank_cr,bank_days,insurance,insurance_class,insurance_cr,insurance_days,financeetc,financeetc_class,financeetc_cr,financeetc_days,corporateetc,corporateetc_class,corporateetc_cr,corporateetc_days,foreigneretc,foreigneretc_class,foreigneretc_cr,foreigneretc_days,open,high,low,close,close_cr,vol,vol_percent,total,retail_ratio,foreigner_ratio,institution_ratio,financial_ratio,invtrust_ratio,pension_ratio,privequity_ratio,bank_ratio,insurance_ratio,financeetc_ratio,corporateetc_ratio,foreigneretc_ratio
12,2020-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.1525,-1,-2566.0,5,-0.1242,-2,696.0,5,1.0592,5,674.0,7,4.7119,2,272.0,5,-6.9130,1,440.0,7,4.7895,6,3883.0,6,0.0086,13,20.0,5,-0.4595,2,119500.0,119500.0,117500.0,118000.0,-0.42,52706.0,0.0330,19858.0,1.9856,-2.9671,-0.9845,-0.6516,-0.0896,-1.2922,0.3505,0.3394,0.1370,0.2216,1.9554,0.0101
30,2020-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.9350,-1,1105.0,5,-0.7030,2,3761.0,6,-19.5271,1,-11.0,5,-0.9864,-3,213.0,5,-0.5714,2,-1000.0,1,36.0370,-3,4318.0,6,-0.0157,16,261.0,6,-0.3830,4,133500.0,136000.0,132500.0,135500.0,1.50,74968.0,0.0470,22691.0,-3.2824,-1.0881,2.3525,0.7439,-0.1838,0.4870,1.6575,-0.0048,0.0939,-0.4407,1.9030,0.1150
69,2020-04-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.6188,5,9097.0,6,-0.5730,7,-4660.0,3,-1.7433,-1,355.0,6,-1.2595,1,-1072.0,5,-1.2384,-1,98.0,5,0.1951,3,-1024.0,4,-4.4020,-1,72.0,5,0.8462,2,97400.0,101500.0,96200.0,99700.0,2.68,278875.0,0.1748,49460.0,2.5524,-1.3112,-1.0489,-2.3231,0.5020,1.8393,-0.9422,0.0718,-0.2167,0.0198,-0.2070,0.0146
96,2020-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.7136,-3,1979.0,5,-1.5428,1,-5930.0,3,-0.3627,-4,49.0,5,-1.0728,1,689.0,5,-3.6000,1,0.0,5,-1.0000,-3,50.0,5,-1.0785,1,1.0,5,-0.9783,4,94900.0,96200.0,94000.0,96000.0,1.59,112859.0,0.0707,41011.0,-3.2455,4.3249,-1.0919,-0.2487,-0.0600,0.4826,-1.4460,0.0119,0.1680,0.0000,0.0122,0.0002
125,2020-07-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.3965,-7,-4046.0,5,1.3015,-2,-1016.0,5,-1.4077,-1,76.0,5,-0.1059,2,-377.0,5,-1.1158,-1,0.0,5,-1.0000,-6,1049.0,5,-1.6467,1,-14.0,5,-1.2500,-1,99900.0,100500.0,98600.0,99200.0,0.20,56539.0,0.0354,18308.0,4.3855,-1.1088,-3.8420,-0.3840,-0.5287,-2.2100,-0.5549,0.0415,-0.2059,0.0000,0.5730,-0.0076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [78]:
# nan columns 내용을 바로 row의 값으로 대체
col_names = df_common.columns
df_merge =  fillna_with_bfill(df_merge, col_names)

In [83]:
df_merge.iloc[10:25]

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,spx_cr,spx_f,spx_f_cr,kospi,kospi_cr,kosdaq,kosdaq_cr,wti,wti_cr,dxy,dxy_cr,krw,krw_cr,bond_usa_10,bond_usa_10_cr,bond_usa_2,bond_usa_2_cr,bond_usa_3m,bond_usa_3m_cr,bond_kor_10,bond_kor_10_cr,bond_kor_2,bond_kor_2_cr,vix,vix_cr,f_g_index,sox,sox_cr,cpi,cpi_anticipated,cpi_previous,gold,gold_cr,fed_rate,fed_rate_fore,fed_rate_prev,fed_rate_ann,fed_rate_imp,bok_rate,fu_usa_date,...,invtrust_cr,invtrust_days,pension,pension_class,pension_cr,pension_days,privequity,privequity_class,privequity_cr,privequity_days,bank,bank_class,bank_cr,bank_days,insurance,insurance_class,insurance_cr,insurance_days,financeetc,financeetc_class,financeetc_cr,financeetc_days,corporateetc,corporateetc_class,corporateetc_cr,corporateetc_days,foreigneretc,foreigneretc_class,foreigneretc_cr,foreigneretc_days,open,high,low,close,close_cr,vol,vol_percent,total,retail_ratio,foreigner_ratio,institution_ratio,financial_ratio,invtrust_ratio,pension_ratio,privequity_ratio,bank_ratio,insurance_ratio,financeetc_ratio,corporateetc_ratio,foreigneretc_ratio
10,2020-01-16,9357.13,1.06,9133.50,0.81,29297.64,0.92,29240.0,0.73,3316.81,0.84,3316.50,0.69,2248.05,0.77,686.52,1.08,58.52,1.23,97.062,0.11,1159.73,0.16,1.809,1.27,1.5657,0.38,1.559,-0.64,1.700,0.53,1.415,1.80,12.32,-0.81,NaN,1903.9,1.70,2.30,2.30,2.10,1552.27,-0.0026,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,8.6386,3,619.0,5,-0.9079,2,1344.0,5,1.1925,3,-254.0,4,0.8143,-3,5131.0,7,8.8484,4,196.0,6,2.6296,4,4317.0,6,0.0940,11,-77.0,5,-8.0000,-1,117000.0,120000.0,116500.0,118500.0,3.04,194518.0,0.1219,43591.0,-1.5659,-2.2206,2.8137,-1.1376,2.3372,0.1420,0.3083,-0.0583,1.1771,0.0450,0.9903,-0.0177
11,2020-01-17,9388.94,0.34,9174.50,0.45,29348.10,0.17,29279.0,0.13,3329.62,0.39,3325.00,0.26,2250.57,0.11,688.41,0.28,58.54,0.03,97.359,0.31,1160.44,0.06,1.825,0.88,1.5654,-0.02,1.561,0.13,1.745,2.65,1.412,-0.21,12.10,-1.79,NaN,1916.6,0.67,2.30,2.30,2.10,1557.30,0.0032,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,-0.8855,4,-2930.0,5,-5.7334,-1,338.0,5,-0.7485,4,118.0,5,-1.4646,1,-46.0,5,-1.0090,-1,76.0,5,-0.6122,5,3850.0,6,-0.1082,12,37.0,5,-1.4805,1,118500.0,120000.0,118000.0,118500.0,0.00,68191.0,0.0427,23557.0,2.6285,-1.9289,-2.3496,-1.8080,0.4954,-1.2438,0.1435,0.0501,-0.0195,0.0323,1.6343,0.0157
12,2020-01-20,9388.94,0.34,9174.50,0.45,29348.10,0.17,29279.0,0.13,3329.62,0.39,3325.00,0.26,2250.57,0.11,688.41,0.28,58.54,0.03,97.359,0.31,1160.44,0.06,1.825,0.88,1.5654,-0.02,1.561,0.13,1.745,2.65,1.412,-0.21,12.10,-1.79,NaN,1916.6,0.67,2.30,2.30,2.10,1557.30,0.0032,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,-1.1525,-1,-2566.0,5,-0.1242,-2,696.0,5,1.0592,5,674.0,7,4.7119,2,272.0,5,-6.9130,1,440.0,7,4.7895,6,3883.0,6,0.0086,13,20.0,5,-0.4595,2,119500.0,119500.0,117500.0,118000.0,-0.42,52706.0,0.0330,19858.0,1.9856,-2.9671,-0.9845,-0.6516,-0.0896,-1.2922,0.3505,0.3394,0.1370,0.2216,1.9554,0.0101
13,2020-01-21,9370.81,-0.19,9169.75,0.00,29196.04,-0.52,29179.0,-0.34,3320.79,-0.27,3319.50,-0.10,2239.69,-1.01,676.52,-1.02,58.34,-0.66,97.294,-0.07,1169.01,0.90,1.774,-2.68,1.5325,-1.84,1.556,-0.17,1.688,-4.47,1.397,-2.99,12.85,6.20,NaN,1915.0,-0.08,2.30,2.30,2.10,1556.56,-0.0030,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,-10.2584,1,-1140.0,5,-0.5557,-3,1940.0,6,1.7874,6,32.0,5,-0.9525,3,428.0,5,0.5735,2,-1.0,5,-1.0023,-1,3411.0,5,-0.1216,14,-2.0,5,-1.1000,-1,118000.0,118500.0,116500.0,117000.0,-0.85,63919.0,0.0401,32706.0,2.3910,-4.6505,1.2172,0.3284,0.5039,-0.3486,0.5932,0.0098,0.1309,-0.0003,1.0429,-0.0006
14,2020-01-22,9383.77,0.14,9195.75,0.28,29186.27,-0.03,29140.0,-0.13,3321.75,0.03,3319.75,0.01,2267.25,1.23,688.25,1.73,56.74,-2.74,97.295,0.00,1164.76,-0.36,1.769,-0.29,1.5283,-0.27,1.554,-0.13,1.723,2.07,1.421,1.72,12.91,0.47,NaN,1930.2,0.80,2.30,2.30,2.10,1561.45,0.0031,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,3.4867,2,-8727.0,5,6.6553,-4,16609.0,9,7.5613,7,123.0,5,2.8438,4,5408.0,7,11.6355,3,414.0,7,-415.0000,1,-240.0,5,-1.0704,-1,56.0,5,-29.0000,1,117500.0,127000.0,117500.0,127000.0,8.55,340289.

In [91]:
dir_name = '../data/analysis/'
company = 'hyunmotor_ml.csv'
df_merge.to_csv(dir_name+company, index=False)

In [4]:
dir_name = '../data/analysis/'
company = 'hyunmotor_ml.csv'
df = pd.read_csv(dir_name+company)

In [5]:
df.head()

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,spx_cr,spx_f,spx_f_cr,kospi,kospi_cr,kosdaq,kosdaq_cr,wti,wti_cr,dxy,dxy_cr,krw,krw_cr,bond_usa_10,bond_usa_10_cr,bond_usa_2,bond_usa_2_cr,bond_usa_3m,bond_usa_3m_cr,bond_kor_10,bond_kor_10_cr,bond_kor_2,bond_kor_2_cr,vix,vix_cr,f_g_index,sox,sox_cr,cpi,cpi_anticipated,cpi_previous,gold,gold_cr,fed_rate,fed_rate_fore,fed_rate_prev,fed_rate_ann,fed_rate_imp,bok_rate,fu_usa_date,...,invtrust_cr,invtrust_days,pension,pension_class,pension_cr,pension_days,privequity,privequity_class,privequity_cr,privequity_days,bank,bank_class,bank_cr,bank_days,insurance,insurance_class,insurance_cr,insurance_days,financeetc,financeetc_class,financeetc_cr,financeetc_days,corporateetc,corporateetc_class,corporateetc_cr,corporateetc_days,foreigneretc,foreigneretc_class,foreigneretc_cr,foreigneretc_days,open,high,low,close,close_cr,vol,vol_percent,total,retail_ratio,foreigner_ratio,institution_ratio,financial_ratio,invtrust_ratio,pension_ratio,privequity_ratio,bank_ratio,insurance_ratio,financeetc_ratio,corporateetc_ratio,foreigneretc_ratio
0,2020-01-02,9092.19,1.33,8891.75,1.31,28868.80,1.16,28841.0,0.96,3257.85,0.84,3259.00,0.66,2175.17,-1.02,674.02,0.63,61.18,-0.24,96.525,0.49,1157.35,0.29,1.877,-2.29,1.5710,0.00,1.533,-1.50,1.638,-2.03,1.335,-2.20,12.47,-9.51,NaN,1887.9,2.07,2.1,2.0,1.8,1528.95,0.0069,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,NaN,-1,-9936.0,4.0,NaN,-1,-22.0,5.0,NaN,-1,50.0,5.0,NaN,1,355.0,5.0,NaN,1,112.0,5.0,NaN,1,4600.0,6.0,NaN,1,40.0,5.0,NaN,1,121000.0,121000.0,118000.0,118000.0,-2.07,66205.0,0.0415,54274.0,4.0498,-2.5887,-2.3160,-0.4787,-0.0978,-1.8307,-0.0041,0.0092,0.0654,0.0206,0.8476,0.0074
1,2020-01-03,9020.77,-0.79,8810.00,-0.92,28634.88,-0.81,28602.0,-0.83,3234.85,-0.71,3235.50,-0.72,2176.46,0.06,669.93,-0.61,63.05,3.06,96.521,0.00,1164.95,0.66,1.793,-4.46,1.5326,-2.44,1.520,-0.85,1.552,-5.25,1.290,-3.37,14.02,12.43,NaN,1854.0,-1.80,2.1,2.0,1.8,1552.24,0.0152,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,0.2938,-2,-3659.0,5.0,-0.6317,-2,-1735.0,5.0,77.8636,-2,-43.0,5.0,-1.8600,-1,-1553.0,5.0,-5.3746,-1,-17.0,5.0,-1.1518,-1,6353.0,7.0,0.3811,2,150.0,5.0,2.7500,2,118500.0,119000.0,115000.0,116000.0,-1.69,133657.0,0.0838,98309.0,4.3385,-2.8420,-2.1579,-1.3755,-0.0699,-0.3722,-0.1765,-0.0044,-0.1580,-0.0017,0.6462,0.0153
2,2020-01-06,9071.47,0.56,8847.50,0.43,28703.38,0.24,28642.0,0.14,3246.28,0.35,3243.50,0.25,2155.07,-0.98,655.31,-2.18,63.27,0.35,96.350,-0.18,1166.94,0.17,1.809,2.37,1.5466,2.11,1.538,0.49,1.537,-1.47,1.299,0.70,13.85,-1.21,NaN,1834.7,-1.04,2.1,2.0,1.8,1563.83,0.0075,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,-0.1805,-3,1346.0,5.0,-1.3679,1,127.0,5.0,-1.0732,1,0.0,5.0,-1.0000,-2,3266.0,7.0,-3.1030,1,-9.0,5.0,-0.4706,-2,3571.0,6.0,-0.4379,3,-7.0,5.0,-1.0467,-1,116000.0,117000.0,115500.0,116000.0,0.00,38907.0,0.0244,21934.0,1.2118,-0.5216,-2.3151,-4.2140,-0.2567,0.6137,0.0579,0.0000,1.4890,-0.0041,1.6281,-0.0032
3,2020-01-07,9068.58,-0.03,8853.00,0.06,28583.68,-0.42,28526.0,-0.40,3237.18,-0.28,3235.25,-0.25,2175.54,0.95,663.44,1.24,62.70,-0.90,96.702,0.37,1167.30,0.03,1.811,0.10,1.5384,-0.53,1.551,0.85,1.616,5.14,1.343,3.39,13.79,-0.43,NaN,1867.3,1.78,2.1,2.0,1.8,1595.24,0.0201,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,2.0870,-4,-6291.0,5.0,-5.6738,-1,81.0,5.0,-0.3622,2,3.0,5.0,inf,1,-755.0,5.0,-1.2312,-1,0.0,5.0,-1.0000,-3,3741.0,6.0,0.0476,4,-51.0,5.0,6.2857,-2,116500.0,117000.0,115500.0,115500.0,-0.43,78229.0,0.0490,34714.0,3.8981,-1.6757,-3.2854,-0.7792,-0.5007,-1.8122,0.0233,0.0009,-0.2175,0.0000,1.0777,-0.0147
4,2020-01-08,9129.24,0.67,8944.50,1.03,28745.09,0.56,28770.0,0.86,3253.05,0.49,3260.25,0.77,2151.31,-1.11,640.94,-3.39,59.61,-4.93,96.996,0.30,1162.25,-0.43,1.874,3.48,1.5846,3.00,1.544,-0.45,1.628,0.74,1.322,-1.56,13.45,-2.47,NaN,1867.6,0.02,2.1,2.0,1.8,1557.89,-0.0234,1.75,1.75,1.75,1.0,1.0,0.5,0.0,...,-0.4166,-5,-16321.0,3.0,1.5943,-2,2514.0,6.0,30.0370,3,2.0,5.0,-0.3333,2,-3.0,5.0,-0.9960,-2,-57.0,5.0,-inf,-4,4562.0,6.0,0.2195,5,0.0,5.0,-1.0000,-3,115000.0,11

In [10]:
col = df.columns[1:]
dff = df[col].astype(float)

In [18]:
ds = df.isin([np.inf, -np.inf])

In [23]:
df[~ds]

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,spx_cr,spx_f,spx_f_cr,kospi,kospi_cr,kosdaq,kosdaq_cr,wti,wti_cr,dxy,dxy_cr,krw,krw_cr,bond_usa_10,bond_usa_10_cr,bond_usa_2,bond_usa_2_cr,bond_usa_3m,bond_usa_3m_cr,bond_kor_10,bond_kor_10_cr,bond_kor_2,bond_kor_2_cr,vix,vix_cr,f_g_index,sox,sox_cr,cpi,cpi_anticipated,cpi_previous,gold,gold_cr,fed_rate,fed_rate_fore,fed_rate_prev,fed_rate_ann,fed_rate_imp,bok_rate,fu_usa_date,...,invtrust_cr,invtrust_days,pension,pension_class,pension_cr,pension_days,privequity,privequity_class,privequity_cr,privequity_days,bank,bank_class,bank_cr,bank_days,insurance,insurance_class,insurance_cr,insurance_days,financeetc,financeetc_class,financeetc_cr,financeetc_days,corporateetc,corporateetc_class,corporateetc_cr,corporateetc_days,foreigneretc,foreigneretc_class,foreigneretc_cr,foreigneretc_days,open,high,low,close,close_cr,vol,vol_percent,total,retail_ratio,foreigner_ratio,institution_ratio,financial_ratio,invtrust_ratio,pension_ratio,privequity_ratio,bank_ratio,insurance_ratio,financeetc_ratio,corporateetc_ratio,foreigneretc_ratio
0,2020-01-02,9092.19,1.33,8891.75,1.31,28868.80,1.16,28841.0,0.96,3257.85,0.84,3259.00,0.66,2175.17,-1.02,674.02,0.63,61.18,-0.24,96.525,0.49,1157.35,0.29,1.877,-2.29,1.5710,0.00,1.533,-1.50,1.638,-2.03,1.335,-2.20,12.47,-9.51,NaN,1887.9,2.07,2.1,2.0,1.8,1528.95,0.0069,1.75,1.75,1.75,1.0,1.0,0.50,0.0,...,NaN,-1,-9936.0,4.0,NaN,-1,-22.0,5.0,NaN,-1,50.0,5.0,NaN,1,355.0,5.0,NaN,1,112.0,5.0,NaN,1,4600.0,6.0,NaN,1,40.0,5.0,NaN,1,121000.0,121000.0,118000.0,118000.0,-2.07,66205.0,0.0415,54274.0,4.0498,-2.5887,-2.3160,-0.4787,-0.0978,-1.8307,-0.0041,0.0092,0.0654,0.0206,0.8476,0.0074
1,2020-01-03,9020.77,-0.79,8810.00,-0.92,28634.88,-0.81,28602.0,-0.83,3234.85,-0.71,3235.50,-0.72,2176.46,0.06,669.93,-0.61,63.05,3.06,96.521,0.00,1164.95,0.66,1.793,-4.46,1.5326,-2.44,1.520,-0.85,1.552,-5.25,1.290,-3.37,14.02,12.43,NaN,1854.0,-1.80,2.1,2.0,1.8,1552.24,0.0152,1.75,1.75,1.75,1.0,1.0,0.50,0.0,...,0.2938,-2,-3659.0,5.0,-0.6317,-2,-1735.0,5.0,77.8636,-2,-43.0,5.0,-1.8600,-1,-1553.0,5.0,-5.3746,-1,-17.0,5.0,-1.1518,-1,6353.0,7.0,0.3811,2,150.0,5.0,2.7500,2,118500.0,119000.0,115000.0,116000.0,-1.69,133657.0,0.0838,98309.0,4.3385,-2.8420,-2.1579,-1.3755,-0.0699,-0.3722,-0.1765,-0.0044,-0.1580,-0.0017,0.6462,0.0153
2,2020-01-06,9071.47,0.56,8847.50,0.43,28703.38,0.24,28642.0,0.14,3246.28,0.35,3243.50,0.25,2155.07,-0.98,655.31,-2.18,63.27,0.35,96.350,-0.18,1166.94,0.17,1.809,2.37,1.5466,2.11,1.538,0.49,1.537,-1.47,1.299,0.70,13.85,-1.21,NaN,1834.7,-1.04,2.1,2.0,1.8,1563.83,0.0075,1.75,1.75,1.75,1.0,1.0,0.50,0.0,...,-0.1805,-3,1346.0,5.0,-1.3679,1,127.0,5.0,-1.0732,1,0.0,5.0,-1.0000,-2,3266.0,7.0,-3.1030,1,-9.0,5.0,-0.4706,-2,3571.0,6.0,-0.4379,3,-7.0,5.0,-1.0467,-1,116000.0,117000.0,115500.0,116000.0,0.00,38907.0,0.0244,21934.0,1.2118,-0.5216,-2.3151,-4.2140,-0.2567,0.6137,0.0579,0.0000,1.4890,-0.0041,1.6281,-0.0032
3,2020-01-07,9068.58,-0.03,8853.00,0.06,28583.68,-0.42,28526.0,-0.40,3237.18,-0.28,3235.25,-0.25,2175.54,0.95,663.44,1.24,62.70,-0.90,96.702,0.37,1167.30,0.03,1.811,0.10,1.5384,-0.53,1.551,0.85,1.616,5.14,1.343,3.39,13.79,-0.43,NaN,1867.3,1.78,2.1,2.0,1.8,1595.24,0.0201,1.75,1.75,1.75,1.0,1.0,0.50,0.0,...,2.0870,-4,-6291.0,5.0,-5.6738,-1,81.0,5.0,-0.3622,2,3.0,5.0,NaN,1,-755.0,5.0,-1.2312,-1,0.0,5.0,-1.0000,-3,3741.0,6.0,0.0476,4,-51.0,5.0,6.2857,-2,116500.0,117000.0,115500.0,115500.0,-0.43,78229.0,0.0490,34714.0,3.8981,-1.6757,-3.2854,-0.7792,-0.5007,-1.8122,0.0233,0.0009,-0.2175,0.0000,1.0777,-0.0147
4,2020-01-08,9129.24,0.67,8944.50,1.03,28745.09,0.56,28770.0,0.86,3253.05,0.49,3260.25,0.77,2151.31,-1.11,640.94,-3.39,59.61,-4.93,96.996,0.30,1162.25,-0.43,1.874,3.48,1.5846,3.00,1.544,-0.45,1.628,0.74,1.322,-1.56,13.45,-2.47,NaN,1867.6,0.02,2.1,2.0,1.8,1557.89,-0.0234,1.75,1.75,1.75,1.0,1.0,0.50,0.0,...,-0.4166,-5,-16321.0,3.0,1.5943,-2,2514.0,6.0,30.0370,3,2.0,5.0,-0.3333,2,-3.0,5.0,-0.9960,-2,-57.0,5.0,NaN,-4,4562.0,6.0,0.2195,5,0.0,5.0,-1.0000,-3,115000.